# Transcript to grapheme dictionary and training file

In [ ]:

import os
from matplotlib.pylab import *
import librosa
import torch
from epoch_time import epoch_time
from tqdm.notebook import tqdm
from txt_to_stm import txt_to_stm
import pandas as pd
import numpy as np
from padarray import padarray
from to_samples import to_samples
from torch.utils.data import TensorDataset, DataLoader
import audioread
import random
import soundfile as sf
from pathlib import Path

In [ ]:
os.getcwd()

In [ ]:

sample_rate=8000
window = sample_rate
H=window

## Grapheme dictionary

In [2]:
from glob import glob
stage='NIST'
transcripts = list(sorted(glob(f'{stage}/*/build/transcription/*.txt')))
len(transcripts)

122

In [6]:
import os
import pandas as pd
from txt_to_stm import txt_to_stm
T=[]
for transcript_file in transcripts:
    file = "_".join(os.path.basename(transcript_file).split("_")[:-1])
    channel = os.path.basename(transcript_file).split("_")[-1].split(".")[-2]
    transcript_df = pd.read_csv(transcript_file, sep = "\n", header = None, names = ["content"])
    result = [x[-1] for x in txt_to_stm(transcript_df, file, channel) if len(x)==6]
    T.append(''.join(result))

In [7]:
vocab=list(set(''.join(T)))

In [8]:
import json
with open('amharic_characters.json','w') as f:
    json.dump(vocab, f)

## Training file

In [9]:
chunks = list(sorted(glob(f'{stage}/*/build/transcription_split/*.txt')))
len(chunks)

10777

In [11]:
L=[]
for text in chunks:
    audio=text.replace('transcription','audio').replace('txt', 'wav')
    L.append(f'{audio},{text}')

In [12]:
L='\n'.join(L)

In [13]:
man_fn='amharic_manifest.csv'
with open(man_fn,'w') as f:
    f.write(L)